In [1]:
# You should only need to run this cell once when you (re)start the kernel. Thereafter, includet should import any changes.
using Pkg
Pkg.activate("../..") # change this to your package-install location

using MLDatasets: MNIST
using ImageCore
using Flux: onehotbatch, onecold

using CUDA
CUDA.allowscalar(false)

using Revise
includet("activations_and_losses.jl")
includet("dense_network_model.jl")
includet("dense_network_training.jl")

  Activating project at `~/Workspace/CSC381`


In [2]:
train_x, train_y = MNIST.traindata()
test_x,  test_y  = MNIST.testdata()

train_set_size = size(train_x)[end]
test_set_size = size(test_x)[end]
image_dimensions = size(train_x)[1:end-1]

println(train_set_size, " points in the training set")
println(test_set_size, " points in the testing set")
println("image inputs have dimension ", image_dimensions)

60000 points in the training set
10000 points in the testing set
image inputs have dimension (28, 28)


In [3]:
image_ID = rand(1:train_set_size)
println("image #", image_ID, ", label = ", train_y[image_ID])
MNIST.convert2image(train_x[:,:,image_ID])

image #48073, label = 9


In [4]:
num_inputs = size(train_x,1) * size(train_x,2)
num_labels = length(unique(train_y))
hidden_sizes = [20,20]
hidden_activations = [ReLU_activation, tanh_activation]
output_activation = softmax_activation
nn1 = DenseNetworkCPU(num_inputs, num_labels, hidden_sizes; hidden_activations=hidden_activations, output_activation=output_activation);
nn2 = DenseNetworkGPU(num_inputs, num_labels, hidden_sizes; hidden_activations=hidden_activations, output_activation=output_activation);

In [25]:
cu(Array{Float32}(reshape(permutedims(train_x, [3,1,2]), 60000, 28*28)))

60000×784 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0 

In [26]:
cu(Array{Float32}(onehotbatch(train_y, 0:9)'))

60000×10 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0


In [7]:
inputs = Array{Float32}(reshape(permutedims(train_x, [3,1,2]), train_set_size, num_inputs))
GPU_inputs = cu(inputs)
targets = onehotbatch(train_y, 0:9)'
GPU_targets = cu(Array{Float32}(targets))
println("input shape: ", size(inputs))
println("target shape: ", size(targets))

input shape: (60000, 784)
target shape: (60000, 10)


In [9]:
train!(nn1, inputs, targets, 0.005, 10, 20; verbose=true)

epoch #1 ... average loss = 0.8664824366569519
epoch #2 ... average loss = 0.5050692558288574
epoch #3 ... average loss = 0.40068140625953674
epoch #4 ... average loss = 0.3451679050922394
epoch #5 ... average loss = 0.3099852204322815
epoch #6 ... average loss = 0.2855943441390991
epoch #7 ... average loss = 0.2640986442565918
epoch #8 ... average loss = 0.2480199933052063
epoch #9 ... average loss = 0.2346649467945099
epoch #10 ... average loss = 0.22376342117786407


10-element Vector{Float64}:
 0.8664824366569519
 0.5050692558288574
 0.40068140625953674
 0.3451679050922394
 0.3099852204322815
 0.2855943441390991
 0.2640986442565918
 0.2480199933052063
 0.2346649467945099
 0.22376342117786407

In [ ]:
train!(nn2, GPU_inputs, GPU_targets, 0.005, 10, 20; verbose=true)

In [10]:
predictions = predict(nn1, inputs)
predicted_labels = onecold(predictions', 0:9)
correct = predicted_labels .== train_y
sum(correct) / 60000

0.9349666666666666

In [12]:
using StatsBase: sample

In [28]:
image_ID = sample((1:train_set_size)[.!correct])
println("image #", image_ID, ", predicted: ", predicted_labels[image_ID], ", actual: ", train_y[image_ID])
MNIST.convert2image(train_x[:,:,image_ID])

image #51159, predicted: 6, actual: 8
